In [1]:
import re
import numpy as np
import math
from matplotlib import pyplot as plt

In [2]:
with open("example.txt") as f:
    data = f.read()
    lines = data.strip().split("\n")
    records = data.split("\n\n")
    example = True

In [3]:
with open("input.txt") as f:
    data = f.read()
    lines = data.strip().split("\n")   
    records = data.split("\n\n")
    example = False

In [4]:
#board = np.array([list(x) for x in lines])
#plt.matshow(screen)

In [5]:
board_raw = records[0]
moves_raw = records[1]

In [6]:
max_len = max(len(l) for l in board_raw.split("\n"))
n_rows = len(board_raw.split("\n"))

In [7]:
board = np.zeros([n_rows,max_len])
mapping = {" ":0,".":1,"#":2}
for i,line in enumerate(board_raw.split("\n")):
    #print(line)
    
    tmp = np.array([mapping[x] for x in line.replace("\n","")])
    #print(tmp)
    
    board[i][:len(tmp)] += tmp

In [8]:
np.array([[0,1,2],[3,4,5]]).shape

(2, 3)

In [9]:
print(board)

[[0. 0. 0. ... 1. 1. 1.]
 [0. 0. 0. ... 1. 1. 1.]
 [0. 0. 0. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]]


In [10]:
def get_str_board(board):
    str_mapping = {0:" ",1:".",2:"#"}
    str_board = []
    for line in board:
        tmp = [str_mapping[x] for x in line]
        str_board.append(tmp)
    return str_board

def print_str_board(str_board):
    tmp = ""
    for line in str_board:
        tmp += "".join(line)+"\n"

    print(tmp)
    
def add_symbol(index,direction_index,s_board):
    global d_symbols
    if s_board[index[0]][index[1]] != " ":
        s_board[index[0]][index[1]] = d_symbols[direction_index]
    # for debug
    s_board[index[0]][index[1]] = d_symbols[direction_index]

    return s_board


In [11]:
# Warpings:
side_length = 4 if example else 50
example_sides = """
0010
2340
0056
"""
real = """
011
010
110
100
"""
real = """
012
030
450
600
"""
# warps go from just past the side to the next side
# they will re-orient the cursor
# this thus is for leaving a side going 1 direction, and the direction you go on the next one
real_warps = {
    (1,"R"):(2,"R"),
    (1,"D"):(3,"D"),
    (1,"L"):(4,"R"),
    (1,"U"):(6,"R"),
    
    (2,"R"):(5,"L"),
    (2,"D"):(3,"L"),
    (2,"L"):(1,"L"),
    (2,"U"):(6,"U"),
    
    (3,"R"):(2,"U"),
    (3,"D"):(5,"D"),
    (3,"L"):(4,"D"),
    (3,"U"):(1,"U"),
    
    (4,"R"):(5,"R"),
    (4,"D"):(6,"D"),
    (4,"L"):(1,"R"),
    (4,"U"):(3,"R"),
                       
    (5,"R"):(2,"L"),
    (5,"D"):(6,"L"),
    (5,"L"):(4,"L"),
    (5,"U"):(3,"U"),
                  
    (6,"R"):(5,"U"),
    (6,"D"):(2,"D"),
    (6,"L"):(1,"D"),
    (6,"U"):(4,"U")
}
d_to_index = {"R":0,"D":1,"L":2,"U":3}
index_to_d = {0:"R",1:"D",2:"L",3:"U"}
swap_d = {"R":"L","L":"R","U":"D","D":"U"}
# crossings are reflexive, so check there are no errors
for k,v in real_warps.items():
    ik,iv = (k[0],swap_d[k[1]]),(v[0],swap_d[v[1]])
    if  ik != real_warps[iv]:
        print("error on",k,v)

# now that we're done with that, we actually do want number directions
warps = {}
for k,v in real_warps.items():
    nk,nv = (k[0],d_to_index[k[1]]),(v[0],d_to_index[v[1]])
    warps[nk] = nv
print(warps)
tmp_real = []
for row in real.strip().split("\n"):
    tmp_real.append([int(x) for x in row])
real = np.array(tmp_real)
print(real)

side_map = None if example else real

print(side_map)


directions = [(0,1),(1,0),(0,-1),(-1,0)]
directions = [np.array(x) for x in directions]

inverted = {("D","U"),("U","D"),("R","L"),("L","R"),("R","D"),("U","L"),("D","R"),("L","U")}
#starts    = [(y,0),(0,x),(y,side_length-1),(side_length-1,x)]
"""
will return index 1 into the new side
"""
def convert_index(index,old_direction,new_direction):
    global invered
    global side_length
    interesting_val = index[0] if old_direction % 2 == 0 else index[1]
    if (index_to_d[old_direction],index_to_d[new_direction]) in inverted:
        interesting_val = side_length-1 - interesting_val
        
    interesting_val %= side_length

    #interesting_val *= -1 if old_direction // 2 else 1
    if new_direction == 0:
        return interesting_val,0
    elif new_direction == 1:
        return 0,interesting_val
    elif new_direction == 2:
        return interesting_val,side_length-1
    elif new_direction == 3:
        return side_length-1,interesting_val
    
    # flips of "interesting_val":
    # 2->5 right to left
    # 1->4 left to right
    # example:
    # 4->6 right to down
    # 2->6 left to up
    # 6->2 down to right
    # 5->2 down to up 
#convert_index(

{(1, 0): (2, 0), (1, 1): (3, 1), (1, 2): (4, 0), (1, 3): (6, 0), (2, 0): (5, 2), (2, 1): (3, 2), (2, 2): (1, 2), (2, 3): (6, 3), (3, 0): (2, 3), (3, 1): (5, 1), (3, 2): (4, 1), (3, 3): (1, 3), (4, 0): (5, 0), (4, 1): (6, 1), (4, 2): (1, 0), (4, 3): (3, 0), (5, 0): (2, 2), (5, 1): (6, 2), (5, 2): (4, 2), (5, 3): (3, 3), (6, 0): (5, 3), (6, 1): (2, 1), (6, 2): (1, 1), (6, 3): (4, 3)}
[[0 1 2]
 [0 3 0]
 [4 5 0]
 [6 0 0]]
[[0 1 2]
 [0 3 0]
 [4 5 0]
 [6 0 0]]


In [12]:
def index_to_square(sq_size,side_map,index):
    y = index[0] // sq_size
    x = index[1] // sq_size
    return side_map[y,x],(y,x)

def square_to_index(sq_size,side_map,square):
    tmp = np.where(side_map == square)
    return (tmp[0][0]*sq_size,tmp[1][0]*sq_size)

print("index to square",index_to_square(side_length,side_map,(0,50)))
print("square_to_index",square_to_index(side_length,side_map,2))

index to square (1, (0, 1))
square_to_index (0, 100)


In [13]:
class Moves():
    moves = ""
    spot = 0
    def __init__(self,move_line):
        self.moves = re.findall(r"(\d+)([RL])",move_line)
        end = re.search(r"(\d+)$",move_line)
        end = end.groups()[0]
        self.moves.append((end,""))
        #print(move_line)
    def get_next(self):
        tmp_move,tmp_dir = self.moves[self.spot]
        self.spot += 1
        return int(tmp_move),tmp_dir
    def __repr__(self):
        return str(self.moves)+","+str(self.spot)

In [14]:

str_board = get_str_board(board)
moves = Moves(moves_raw)
next_move = moves.get_next()
directions = [(0,1),(1,0),(0,-1),(-1,0)]
directions = [np.array(x) for x in directions]
d_symbols  = [">",  "v",  "<",   "^"]
d_direction_index = {"R":1,"L":-1}
direction_index = 0
index = np.array([0,np.where(board[0,:] == 1)[0][0]])
draw_picture = True
debug = False
part2 = True
if debug:
    print("state is",index,direction_index)
    print("doing move",next_move)
    print_str_board(str_board)
    
    
debug_sus = []
    
# while next_move[1] != "":
while True:
    for i in range(next_move[0]):
        next_index = index + directions[direction_index]
        next_index = (next_index[0]%board.shape[0],next_index[1]%board.shape[1])

        # skip any empty spaces found
        while board[tuple(next_index)] == 0:
            if part2:
                if debug:
                    str_board = add_symbol(next_index,direction_index,str_board)

                # find where along the edge we are now
                y,x = index[0]%side_length,index[1]%side_length
                if debug:
                    print("location on side",y,x)
                
                # find what side we're on
                current_side,_ = index_to_square(side_length,side_map,index)
                if debug:
                    print("current_side",current_side)
                
                # find which new side we'll be on
                new_side,new_direction = warps[(current_side,direction_index)]
                
                # find where along the new side we'll be on
                loc_in_new_side = convert_index(index,direction_index,new_direction)

                # find the corrdinate 
                start_coord = square_to_index(side_length,side_map,new_side)
                new_coord = np.array(start_coord) + np.array(loc_in_new_side)
                #print(start_coord)
                #print(loc_in_new_side)
                #print(new_coord)
                if board[tuple(new_coord)] == 1:
                    if debug:
                        print("moved from",current_side," to ",new_side)
                        print("at",index,"->",new_coord)
                    direction_index = new_direction
                    next_index = new_coord
                elif board[tuple(new_coord)] == 2:
                    break
                else:
                    print("we broke something")
                    print(index)
                    print(new_coord)
                    print("should have been side",new_side)
                    print("was on",current_side)
                if debug:
                    tmp = next_index-directions[direction_index]
                    if tmp[0] < board.shape[0] and tmp[1] < board.shape[1]:
                        str_board = add_symbol(tmp,direction_index,str_board)

                    str_board = add_symbol(next_index,direction_index,str_board)

            else:
                if draw_picture:
                    str_board = add_symbol(next_index,direction_index,str_board)
                next_index += directions[direction_index]
                next_index = (next_index[0]%board.shape[0],next_index[1]%board.shape[1])
            
        if board[tuple(next_index)] == 1:
            index = next_index
            if draw_picture:
                str_board = add_symbol(next_index,direction_index,str_board)
    
    if next_move[1] == "":
        break
    direction_index += d_direction_index[next_move[1]]
    direction_index %= 4
    if draw_picture:
        str_board = add_symbol(index,direction_index,str_board)
 
        # if 2, it doesn't move
    if debuh:
        print("### index",moves.spot,next_move)
        print("state is",index,direction_index)
        #print("doing move",next_move)
        #print_str_board(str_board)
    
    next_move = moves.get_next()
print("final_move is",next_move)


final_move is (14, '')


In [15]:
#print(np.where(board[:,0] != 1)[0])
#print(board[:,2])
print(board[(119,84)])

1.0


In [16]:
print(index)

(114, 76)


In [17]:
print(board.shape)
print(1000 * (1+index[0]) + 4 * (1+index[1]) + direction_index)
# guessed 28404, too low
# swine 1 indexing in the example. it was actually 29408
# part 2, guessed 87332, too low

(200, 150)
115311


In [ ]:
print_str_board(str_board)
